In [2]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import PowerTransformer

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.svm import LinearSVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, make_scorer, roc_auc_score

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

In [3]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv('data/CoffeeHouse_data_encoded.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE()

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)
sm_vc = y_sm.value_counts()
print(f"After SMOTE: {100* sm_vc[1]/(sm_vc.sum()):.2f}% Yes")

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

Orig: 49.92% Yes
After SMOTE: 50.00% Yes


In [4]:
X_train.shape

(3201, 55)

## Testing RandomForestClassifier with Grid Search Cross-validation


In [5]:
roc_auc_scorer = make_scorer(roc_auc_score, average='weighted')

In [6]:
cols_to_view = ['rank_test_score', 'param_max_depth', 'param_max_features', 'param_min_samples_split', 
                'param_min_samples_leaf', 'param_n_estimators', 
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

In [7]:
param_grid={'max_depth': [None, 5, 10, 15],
            'max_features': [5, 6, 7, 8, 9, 10],
            'min_samples_split': [2, 3, 4, 5],
            'min_samples_leaf': [1, 3, 5],
            'n_estimators': [100, 150, 200]}

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring=roc_auc_scorer, cv=5, return_train_score=True, n_jobs=4)
grid_search1.fit(X_train.values, y_train.values)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=4,
             param_grid={'max_depth': [None, 5, 10, 15],
                         'max_features': [5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True,
             scoring=make_scorer(roc_auc_score, average=weighted))

In [8]:
cv_results1 = pd.DataFrame(grid_search1.cv_results_)
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_depth,param_max_features,param_min_samples_split,param_min_samples_leaf,param_n_estimators,mean_test_score,std_test_score,mean_train_score,std_train_score
110,1,None,8,2,1,200,0.76899954,0.01588181,0.99922314,0.00000351
1,2,None,5,2,1,150,0.76713222,0.02031054,0.99922033,0.00000284
41,3,None,6,3,1,200,0.76613221,0.01950003,0.99687987,0.00118282
147,4,None,9,3,1,100,0.76547299,0.01523921,0.99664926,0.00145525
76,5,None,7,3,1,150,0.76515158,0.01745697,0.99594125,0.00072014
150,6,None,9,4,1,100,0.76510271,0.01654888,0.98750391,0.00183353
2,7,None,5,2,1,200,0.76490932,0.01874245,0.99922180,0.00000344
74,8,None,7,2,1,200,0.76489670,0.02381884,0.99922320,0.00000344
40,9,None,6,3,1,150,0.76487128,0.01990970,0.99618053,0.00108805
113,10,None,8,3,1,200,0.76484099,0.02389226,0.99704467,0.00094221


In [9]:
grid_search1.best_estimator_

RandomForestClassifier(max_features=8, n_estimators=200, oob_score=True,
                       random_state=42)

In [10]:
best_model = grid_search1.best_estimator_
best_model.oob_score_

0.767260231177757

In [11]:
cross_val_scores = cross_val_score(best_model, X_train, y_train, scoring=roc_auc_scorer, cv=5)
print(f"(weighted avg) AUC: {np.mean(cross_val_scores)} {np.std(cross_val_scores)}")

(weighted avg) AUC: 0.7689995417123742 0.015881813634078458


In [13]:
## fit best model on full train and test on test data
best_model = best_model.fit(X_train, y_train)

## use trained model to get predicted target var
# y_train_pred = best_model.predict(X_train)   
y_test_pred = best_model.predict(X_test) 

# print(f"(weighted avg) AUC: train {roc_auc_score(y_train, y_train_pred, average='weighted')}")
print(f"(weighted avg) AUC: TEST {roc_auc_score(y_test, y_test_pred, average='weighted')}")

(weighted avg) AUC: TEST 0.7869155377988638


In [12]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
14,CoffeeHouse,0.15086089
10,income,0.06711129
8,age,0.05905937
7,time,0.04926985
9,education,0.04785758
13,Bar,0.04770870
16,RestaurantLessThan20,0.04453074
15,CarryAway,0.04448611
12,minsToCouponDest,0.04419782
17,Restaurant20To50,0.04405273


# Attempting to look at the resulting decision tree nodes